In [66]:
from ib.ext.Contract import Contract
from ib.ext.ScannerSubscription import ScannerSubscription
from ib.ext.ContractDetails import ContractDetails
from ib.ext.Order import Order
from ib.opt import Connection, message

from time import sleep 
from datetime import datetime
import pandas as pd
# import backtrader as bt
# import pytz, tzlocal
from bs4 import BeautifulSoup
import re
from ibapi.ticktype import TickTypeEnum
import talib

In [67]:
def error_handler(msg):
    """Handles the capturing of error messages"""
    print ("Server Error: {}".format(msg))

def reply_handler(msg):
    """Handles of server replies"""
    print ("Server Response: {}, {}".format(msg.typeName, msg))

def next_id(msg):
    global order_id
    order_id = int(re.findall('\d+', str(msg))[0])
    
    return order_id
    
def historical_data_handler(msg):  
    global historical_data
    
    if ('finished' in str(msg.date)) == False:   
        
        historical_data.loc[len(historical_data)] = ([msg.date,
                                                      msg.open, msg.high, msg.low, msg.close, msg.volume])
    else:  
        historical_data.set_index('Date',inplace=True)

        
def ReportsFinStatements(msg):
    global annual
    global quarter
   
    parser = BeautifulSoup(msg.data, 'lxml')

    mapp = dict()
    for item in parser.findAll('mapitem'):
        mapp[item['coaitem']] = item.text
        
    
    #ANNUAL
    index = [];columns = []
    for item in parser.findAll('annualperiods')[0].findAll('fiscalperiod'):
        index.append(datetime.strptime(item['fiscalyear'], '%Y').year)
    
    for i in parser.findAll('annualperiods')[0].findAll('fiscalperiod')[0].findAll('lineitem'):
        columns.append(mapp[i['coacode']])

    annual = pd.DataFrame(index = index, columns = columns)
    annual.index.name = 'Annual'
    for item in parser.findAll('annualperiods')[0].findAll('fiscalperiod'):
        for i in item.findAll('lineitem'):
            try:
                annual.loc[datetime.strptime(item['fiscalyear'], '%Y').year,mapp[i['coacode']]] = float(i.text)
            except:
                annual.loc[datetime.strptime(item['fiscalyear'], '%Y').year,mapp[i['coacode']]] = np.nan
        
    # QUARTER
    index_q = [];columns_q = []
    for item in parser.findAll('interimperiods')[0].findAll('fiscalperiod'):
        index_q.append(datetime.strptime(item['enddate'], '%Y-%m-%d').date())
    
    for i in parser.findAll('interimperiods')[0].findAll('fiscalperiod')[0].findAll('lineitem'):
        columns_q.append(mapp[i['coacode']])

    quarter = pd.DataFrame(index = index_q, columns = columns_q)
    quarter.index.name = 'Quarter'
    for item in parser.findAll('interimperiods')[0].findAll('fiscalperiod'):
        for i in item.findAll('lineitem'):
            try:
                quarter.loc[datetime.strptime(item['enddate'], '%Y-%m-%d').date(),mapp[i['coacode']]] = float(i.text)
            except:
                quarter.loc[datetime.strptime(item['enddate'], '%Y-%m-%d').date(),mapp[i['coacode']]] = np.nan


def position_account(msg):
    global account
    account = msg

def Scanner(msg):
    global symbol
    global sectype
    global exchange
    global currency
        
    symbol.append(msg.contractDetails.m_summary.m_symbol)
    sectype.append(msg.contractDetails.m_summary.m_secType)
    exchange.append(msg.contractDetails.m_summary.m_exchange)
    currency.append(msg.contractDetails.m_summary.m_currency)

def Analyst_Estimates(msg):
    global resc
    global resc_ann
    global resc_q
    
    parser = BeautifulSoup(msg.data, 'lxml')
    resc.iloc[:,0] = parser.find('name').text    
    resc.iloc[:,1] = parser.find('exchange').text
    resc.iloc[:,2] = parser.findAll('secid')[0].text
    resc.iloc[:,3] = parser.findAll('sector')[0].text

    resc.iloc[:,4] = parser.findAll('marketdataitem')[0].text
    resc.iloc[:,5] = parser.findAll('marketdataitem')[1].text
    resc.iloc[:,6] = parser.findAll('marketdataitem')[2].text
    resc.iloc[:,7] = parser.findAll('marketdataitem')[3].text
    resc.iloc[:,8] = parser.findAll('marketdataitem')[4].text
    resc.iloc[:,9] = parser.findAll('fyactual')[0].text
    
    annual = [];columns = [];quarter = []
    for item in parser.findAll('fyactual'):
        columns.append(item['type'])
        for per in item.findAll('fyperiod'):
            if per['periodtype'] == 'A':
                annual.append(per['fyear'])
            if per['periodtype'] == 'Q':
                quarter.append('{}-{}'.format(per['endcalyear'],per['endmonth']))
    
    index = list(set(annual))
    index_q = list(set(quarter))
    resc_ann = pd.DataFrame(index = index, columns = columns).sort_index(axis = 0,ascending = False)
    resc_q = pd.DataFrame(index = index_q, columns = columns).sort_index(axis = 0,ascending = False)
    
    for item in parser.findAll('fyactual'):
        for per in item.findAll('fyperiod'):
            
            if per['periodtype'] == 'Q':
                try:
                    resc_q.loc['{}-{}'.format(per['endcalyear'],per['endmonth']),item['type']] = float(per.find('actvalue').text)
                except:
                    resc_q.loc['{}-{}'.format(per['endcalyear'],per['endmonth']),item['type']] = np.nan  
                    
            if per['periodtype'] == 'A':
                try:
                    resc_ann.loc[per['fyear'],item['type']] = float(per.find('actvalue').text)
                except:
                    resc_ann.loc[per['fyear'],item['type']] = np.nan

                    
def ReportsFinSummary(msg):
    global fin
    
    parser = BeautifulSoup(msg.data, 'lxml')
    date_div = [];data_div = []
    
    for i in parser.findAll('dividendpershare'):
        if i['period'] == '12M' and i['reporttype'] == 'TTM':
            
            date_div.append(i['asofdate'])
            try:
                data_div.append(float(i.text))
            except:
                data_div.append(np.nan)
                
    data_rev = []
    for i in parser.findAll('totalrevenue'):
        if i['period'] == '12M' and i['reporttype'] == 'TTM':
            try:
                data_rev.append(float(i.text))
            except:
                data_rev.append(np.nan)
                
    data_eps = []
    for i in parser.findAll('eps'):
        if i['period'] == '12M' and i['reporttype'] == 'TTM':
            try:
                data_eps.append(float(i.text))
            except:
                data_eps.append(np.nan)

    fin = pd.DataFrame(index = date_div,
                       data = {'Dividend Per Share(TTM)':data_div,
                               'Total Revenue(TTM)':data_rev, 
                               'EPS(TTM)':data_eps}).sort_index(axis = 0,ascending = False)

def ReportSnapshot(msg):
    global business_summary
    global brief
    global snap
    global address
    global ratio
    global estimate
    
    parser = BeautifulSoup(msg.data, 'lxml')
    
    business_summary = parser.findAll('text')[0].text
    brief = parser.findAll('text')[1].text
    address = '{},{},{},{}'.format(parser.find('contactinfo').find('streetaddress').text,
                                   parser.find('contactinfo').find('city').text,
                                   parser.find('contactinfo').find('state-region').text,
                                   parser.find('contactinfo').find('country').text)
    
    snap = pd.DataFrame(index = [parser.findAll('coid')[1].text],
                        data = {'Company Type': parser.find('cotype').text,
                                'Desc':parser.findAll('issue')[0]['desc'],
                                'Exchange': parser.findAll('exchange')[0].text,
                                'Industry':parser.findAll('industry')[0].text,
                                'Index':parser.find('indexconstituet').text})
    dicted = {};dicted_est = {}
    for i in range(len(parser.find('ratios').findAll('ratio'))):
        try:
            dicted[parser.find('ratios').findAll('ratio')[i]['fieldname']] = float(parser.find('ratios').findAll('ratio')[i].text)
        except:
            dicted[parser.find('ratios').findAll('ratio')[i]['fieldname']] = parser.find('ratios').findAll('ratio')[i].text
            
    ratio = pd.DataFrame(index = [parser.findAll('coid')[1].text], data = dicted)
    
    for i in range(len(parser.find('forecastdata').findAll('ratio'))):
        try:
            dicted_est[parser.find('forecastdata').findAll('ratio')[i]['fieldname']] = float(parser.find('forecastdata').findAll('ratio')[i].text)
        except:
            dicted_est[parser.find('forecastdata').findAll('ratio')[i]['fieldname']] = parser.find('forecastdata').findAll('ratio')[i].text
            
    estimate = pd.DataFrame(index = [parser.findAll('coid')[1].text], data = dicted_est)
    
def create_contract(symbol, sec_type, exch, prim_exch, curr):

    contract = Contract()
    contract.m_symbol = symbol
    contract.m_secType = sec_type
    contract.m_exchange = exch
    contract.m_primaryExch = prim_exch
    contract.m_currency = curr
    
    return contract

def create_order(order_type, quantity, action):

    order = Order()
    order.m_orderType = order_type
    order.m_totalQuantity = quantity
    order.m_action = action
    
    return order

def create_scanner(numberOfRows, instrument, locationCode, ScanCode, 
                   AbovePrice, marketCapAbove, AboveVolume):

    scanSub = ScannerSubscription()
    scanSub.numberOfRows(numberOfRows)
    scanSub.m_instrument = instrument
    scanSub.m_locationCode = locationCode
    scanSub.m_scanCode = ScanCode
    scanSub.m_abovePrice = AbovePrice
    scanSub.m_marketCapAbove = marketCapAbove
    scanSub.m_aboveVolume = AboveVolume
    
    return scanSub



In [68]:
#price = pd.DataFrame(columns = ['Ticker Id','Field','Price'])
#size = pd.DataFrame()
#scan_param = None
#tws_conn.register(ScannerParameters, message.scannerParameters)
#tws_conn.register(mktdata, message.tickPrice)
#tws_conn.reqMarketDataType(4) # switch to delayed frozen data if live is not available
#tws_conn.reqMktData(0,contract,"",True)                    
#tws_conn.reqScannerParameters()

# Connecting to TWS

In [69]:
tws_conn = Connection.create(host='127.0.0.1',port=7497, clientId=0)
tws_conn.connect()

Server Version: 76
TWS Time at connection:20191020 10:56:25 EST


True

In [70]:
tws_conn.register(error_handler, 'Error')
tws_conn.register(reply_handler)
#tws_conn.unregister(error_handler, 'Error')
#tws_conn.unregister(reply_handler)

False

# Step 1 : Filtertation

In [71]:
symbol = [];sectype = [];exchange = []; currency = []

In [72]:
ScanCode = 'LOW_VS_52W_HL'
scanSub = create_scanner(numberOfRows = 10, instrument = 'STK', 
                         locationCode = 'STK.US.MAJOR', ScanCode = ScanCode, 
                         AbovePrice = '100',marketCapAbove = '100000',
                        AboveVolume = '1000')

In [73]:
tws_conn.register(Scanner, message.scannerData)
tws_conn.reqScannerSubscription(0,scanSub)
sleep(5)
tws_conn.unregister(Scanner,message.scannerData)

True

In [74]:
scanner = pd.DataFrame()
scanner['Symbol'] = symbol
scanner['SecType'] = sectype
scanner['Exchange'] = exchange
scanner['Currency'] = currency
scanner.index.name = ScanCode 

In [75]:
scanner

,Symbol,SecType,Exchange,Currency
LOW_VS_52W_HL,,,,
0,BIIB,STK,SMART,USD
1,ICUI,STK,SMART,USD
2,MED,STK,SMART,USD
3,ADS,STK,SMART,USD
4,SPG,STK,SMART,USD
5,ABMD,STK,SMART,USD
6,BIDU,STK,SMART,USD
7,JAZZ,STK,SMART,USD
8,ULTA,STK,SMART,USD


# Step 2 : Iterate over selceted tickers and collect data

## ReportsFinStatements: Financial statements

In [76]:
contract = create_contract(symbol = 'ACN',
                           sec_type = 'STK',
                           exch = 'SMART',
                           prim_exch = 'NASDAQ',
                           curr = 'USD')

In [77]:
annual = pd.DataFrame()
quarter = pd.DataFrame()

tws_conn.register(ReportsFinStatements, message.fundamentalData)
tws_conn.reqFundamentalData(0,contract=contract,reportType='ReportsFinStatements')
sleep(5)
tws_conn.unregister(ReportsFinStatements, message.fundamentalData)

True

## RESC: Analyst estimates

In [78]:
resc = pd.DataFrame(index = [0],columns = ['Name', 'Exchange', 'Symbol', 'Sector', 'CLPRICE', 'SHARESOUT',
                                          'MARKETCAP', '52WKHIGH', '52WKLOW','NAV'])

resc_ann = pd.DataFrame()
resc_q = pd.DataFrame()

tws_conn.register(Analyst_Estimates, message.fundamentalData)
tws_conn.reqFundamentalData(0,contract=contract,reportType='RESC')
sleep(5)
tws_conn.unregister(Analyst_Estimates, message.fundamentalData)

True

## ReportsFinSummary: Financial summary

In [79]:
fin = None
tws_conn.register(ReportsFinSummary, message.fundamentalData)
tws_conn.reqFundamentalData(0,contract=contract,reportType='ReportsFinSummary')
sleep(5)
tws_conn.unregister(ReportsFinSummary, message.fundamentalData)

True

## ReportSnapshot: Company overview

In [80]:
snap = pd.DataFrame()
business_summary = None
brief = None
address = None
ratio = pd.DataFrame()
estimate = pd.DataFrame()

tws_conn.register(ReportSnapshot, message.fundamentalData)
tws_conn.reqFundamentalData(0,contract=contract,reportType='ReportSnapshot')
sleep(5)
tws_conn.unregister(ReportSnapshot, message.fundamentalData)

True

# Display Data

In [81]:
snap

,Company Type,Desc,Exchange,Index,Industry
Accenture Plc,Equity Issue,Common Stock,New York Stock Exchange,S&P 500,IT Services & Consulting - NEC


In [82]:
business_summary

'Accenture plc is a professional services company serving clients in various industries and in geographic regions, including North America, Europe and Growth Markets. It provides management and technology consulting services. Its segments include Communications, Media and Technology; Financial Services; Health and Public Service; Products, and Resources. The Communications, Media & Technology segment serves communications, electronics, technology, media and entertainment industries. The Financial Services segment serves banking, capital markets and insurance industries. The Health & Public service segment serves healthcare payers and providers, and government departments and agencies, public service organizations, educational institutions and non-profit organizations. The Resources segment serves chemicals, energy, forest products, metals and mining, utilities and related industries. It also offers digital advertising services.'

In [117]:
address

'1 Grand Canal Square,DUBLIN,,Ireland'

In [121]:
quarter.loc[quarter.index[0],'Revenue']

11055.65

In [120]:
int(datetime.strptime(str(quarter.index[0]), "%Y-%m-%d").timestamp())==int(datetime.strptime('20190831', '%Y%m%d').timestamp())

True

In [94]:
scanner

,Symbol,SecType,Exchange,Currency
LOW_VS_52W_HL,,,,
0,BIIB,STK,SMART,USD
1,ICUI,STK,SMART,USD
2,MED,STK,SMART,USD
3,ADS,STK,SMART,USD
4,SPG,STK,SMART,USD
5,ABMD,STK,SMART,USD
6,BIDU,STK,SMART,USD
7,JAZZ,STK,SMART,USD
8,ULTA,STK,SMART,USD


In [95]:
annual

,Revenue,Total Revenue,"Cost of Revenue, Total",Gross Profit,"Selling/General/Admin. Expenses, Total",Total Operating Expense,Operating Income,"Interest Inc.(Exp.),Net-Non-Op., Total","Other, Net",Net Income Before Taxes,...,Deferred Income Tax,"Common Stock, Total",Additional Paid-In Capital,Retained Earnings (Accumulated Deficit),Treasury Stock - Common,Unrealized Gain (Loss),Deferred Taxes,Cash Taxes Paid,Cash Interest Paid,"Issuance (Retirement) of Debt, Net"
Annual,,,,,,,,,,,,,,,,,,,,,
2019,43215,43215,29900.3,13314.7,7009.61,36909.9,6305.07,64.545,-117.822,6251.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,41603.4,41603.4,29160.5,12442.9,6601.87,35762.4,5841.04,-77.278,44.33,5808.09,...,125.729,0.072,4870.764,7952.413,-2116.948,2.391,94.000,1373.244,19.673,-4.195
2017,36765.5,36765.5,25735,11030.5,5888.09,32133.1,4632.36,89.143,-105.468,4616.03,...,137.098,0.071,3516.399,7081.855,-1649.090,1.243,-364.133,1288.788,15.751,-2.120
2016,34797.7,34797.7,24520.2,10277.4,5466.98,29138.4,5659.27,-70.067,14.371,5603.57,...,111.020,0.072,2924.729,7879.960,-2591.907,-0.264,65.940,1425.480,16.285,-1.059
2015,32914.4,32914.4,23105.2,9809.24,5308.99,28478.6,4435.87,19.413,-44.752,4410.53,...,91.372,0.076,4516.810,13470.008,-11472.400,-1.561,-459.109,1433.538,14.810,0.701
2014,31874.7,31874.7,22190.2,9684.47,5401.97,27574.2,4300.51,12.749,-15.56,4297.7,...,198.734,0.076,3347.392,11758.131,-9423.202,0.000,-74.092,962.976,17.595,0.543


In [96]:
resc_ann

,OPROFIT as of Aug31 2019,NAV as of Aug31 2019,CAPEX as of Aug31 2019,NPROFIT as of Aug31 2019,ROA as of Aug31 2019,CPS as of Aug31 2019,EPS as of Aug31 2019,REVENUE as of Aug31 2019,EBIT as of Aug31 2019,BVPS as of Aug31 2019,NPROFITREP as of Aug31 2019,PPROFIT as of Aug31 2019,EPSREP as of Aug31 2019,PPROFITREP as of Aug31 2019,EBITDA as of Aug31 2019,NDEBT as of Aug31 2019,ROE as of Aug31 2019,DPS as of Aug31 2019
2018,NaN,6.693,527.504,3759.97,18.72,7.9045,5.742,33713.8,4976.16,13.9359,3539.74,4990.2,5.4012,4948.09,5765.71,-4293.33,45.7,2.2661
2017,NaN,5.4396,437.814,3314.55,18.04,6.39,5.0153,29574.2,4363.89,12.3286,3050.14,4349.83,4.6164,3917.21,5044.3,-3483.38,47.33,2.0536
2016,NaN,5.766,442.535,3175.02,18.35,6.1047,4.759,29304.8,4287.02,10.8155,3838.76,4076.97,5.7482,4993.85,4936.74,-4350.12,52.05,1.9606
2015,4007.7,9.0992,351.782,2913.57,18.08,5.37,4.2925,27649.7,4007.7,8.2999,2878.71,3985.13,4.239,3927.8,4582.93,-3861.23,55.14,1.8167
2014,3343.18,3.9491,250.219,2432.15,17.98,3.9103,3.5138,23323.6,3343.18,7.0577,2432.15,3340.99,3.5138,3340.99,3825.73,-3807.01,58.52,1.4459
2013,3015.14,5.9484,274.157,2224.89,17.89,3.4344,3.1229,21187.3,3015.14,5.9484,2608.3,3016.66,3.657,3218.8,3455.04,-4160.5,65.88,1.2017


In [97]:
resc_q

,OPROFIT as of Aug31 2019,NAV as of Aug31 2019,CAPEX as of Aug31 2019,NPROFIT as of Aug31 2019,ROA as of Aug31 2019,CPS as of Aug31 2019,EPS as of Aug31 2019,REVENUE as of Aug31 2019,EBIT as of Aug31 2019,BVPS as of Aug31 2019,NPROFITREP as of Aug31 2019,PPROFIT as of Aug31 2019,EPSREP as of Aug31 2019,PPROFITREP as of Aug31 2019,EBITDA as of Aug31 2019,NDEBT as of Aug31 2019,ROE as of Aug31 2019,DPS as of Aug31 2019
2019-05,NaN,NaN,123.382,1100.62,NaN,2.8765,1.6977,9763.95,1511.21,18.9646,1100.62,1499.21,1.6977,1499.21,1705.88,-4177.07,36.97,1.2843
2019-02,NaN,NaN,124.597,1001.25,NaN,1.8583,1.5382,9295.08,1232.89,18.6498,1001.25,1226.61,1.5382,1226.61,1428.14,-3960.7,34.63,0
2018-11,NaN,NaN,68.2997,1122.29,NaN,1.389,1.7231,9323.55,1432.1,17.382,1122.29,1407.27,1.7231,1415.81,1618.19,-3817.39,44.32,1.2835
2018-08,NaN,NaN,152.822,878.388,NaN,2.7666,1.346,8646.91,1237.57,13.9359,878.388,1251.49,1.346,1240.93,1437.85,-4293.33,40.91,0
2018-05,NaN,NaN,150.044,1013.05,NaN,2.6282,1.5475,8917.63,1400.3,13.2161,907.771,1394.1,1.3832,1394.1,1606.4,-3374.53,48.12,1.1498
2018-02,NaN,NaN,108.046,843.764,NaN,1.1463,1.2846,7793.06,1042.9,12.7431,732.607,1012.03,1.1138,1012.03,1222.3,-2902.17,44.2,0
2017-11,NaN,NaN,112.316,987.785,NaN,1.2886,1.5076,8020.93,1251.48,12.4577,987.785,1258.43,1.5076,1258.43,1447.42,-3080.27,51.98,1.1202
2017-08,NaN,NaN,162.208,826.671,NaN,2.5034,1.2559,7764.75,1100.28,12.3286,826.671,1096.4,1.2559,1096.4,1297.21,-3483.38,45.53,0
2017-05,NaN,NaN,121.803,900.507,NaN,2.4395,1.3632,7952.49,1233.39,11.8547,621.068,1241.43,0.9417,784.214,1407.81,-3010.91,49.79,1.0852
2017-02,NaN,NaN,96.854,813.274,17.31,0.2134,1.2338,7715.8,1056.26,11.9146,813.274,1037.57,1.2338,1037.57,1230.48,-2981.32,45.6,0


In [98]:
resc['Sector'][0]

'Industrials'

In [90]:
ratio

,EV,Employees,MKTCAP,NHIG,NLOW,NPRICE,PDATE,PEEXCLXOR,PRICE2BK,QBVPS,...,TTMDIVSHR,TTMEBITD,TTMEPSXCLX,TTMGROSMGN,TTMNIAC,TTMPR2REV,TTMREV,TTMREVPS,TTMROEPCT,VOL10DAVG
Accenture Plc,118911.292,459000.0,125018.8,202.8,132.63,186.04,2019-10-17T00:00:00,25.27996,8.68482,21.42128,...,2.92,6993.521,7.35919,30.81033,4779.112,2.89295,43215.01,66.45541,38.58247,1.80782


In [91]:
estimate

,ConsRecom,ProjDPS,ProjEPS,ProjEPSQ,ProjLTGrowthRate,ProjPE,ProjProfit,ProjSales,ProjSalesQ,TargetPrice
Accenture Plc,2.12,3.23,7.808,1.9954,8.716,23.82684,5032.1679,45890.7347,11141.7794,201.4783


## Technical Data

In [28]:
historical_data = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])

In [29]:
tws_conn.register(historical_data_handler, message.historicalData)
tws_conn.reqHistoricalData(0, contract,'','1 M','1 hour','TRADES',1,1)
sleep(5)
tws_conn.unregister(historical_data_handler, message.historicalData)

True

In [30]:
historical_data['MA30'] = talib.SMA(historical_data['Close'], timeperiod=30)
historical_data['MA10'] = talib.SMA(historical_data['Close'], timeperiod=10)
historical_data['RSI'] = talib.RSI(historical_data['Close'], timeperiod=14)
historical_data['Return'] = (historical_data['Close'] - historical_data['Close'].shift(1))/historical_data['Close']
historical_data['macd'], historical_data['macdsignal'], historical_data['macdhist'] = talib.MACDEXT(historical_data['Close'], fastperiod=12, fastmatype=1, slowperiod=26, slowmatype=1, signalperiod=9, signalmatype=1)
historical_data['Beta'] = talib.BETA(historical_data['High'],historical_data['Low'], timeperiod=5)



In [31]:
historical_data.tail(10)

,Open,High,Low,Close,Volume,MA30,MA10,RSI,Return,macd,macdsignal,macdhist,Beta
Date,,,,,,,,,,,,,
20191010 13:00:00,229.89,229.92,229.10,229.74,16295,227.488000,228.470,64.126358,-0.000696,1.209469,0.985872,0.223597,0.156328
20191010 14:00:00,229.75,229.91,228.30,229.77,26596,227.615333,228.776,64.247424,0.000131,1.219492,1.032596,0.186896,2.048586
20191010 15:00:00,229.77,230.30,229.72,230.09,39154,227.741333,229.073,65.581709,0.001391,1.238974,1.073872,0.165102,2.467842
20191011 09:30:00,232.95,233.82,232.30,233.67,48011,227.967000,229.755,76.257254,0.015321,1.525703,1.164238,0.361465,0.784240
20191011 10:00:00,233.70,234.40,233.66,233.94,45068,228.198000,230.418,76.840688,0.001154,1.754499,1.282290,0.472209,0.712736
20191011 11:00:00,233.95,235.20,233.93,234.48,43546,228.445667,231.166,78.004822,0.002303,1.956838,1.417200,0.539638,0.700329
20191011 12:00:00,234.49,234.87,233.08,234.83,28129,228.648000,231.663,78.750390,0.001490,2.120986,1.557957,0.563029,0.709122
20191011 13:00:00,234.82,235.43,234.47,235.30,26290,228.844333,232.179,79.743372,0.001997,2.262914,1.698948,0.563966,0.735259
20191011 14:00:00,235.31,236.56,234.91,236.49,39656,229.080667,232.821,82.032696,0.005032,2.443252,1.847809,0.595443,0.970188


Server Error: <error id=0, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>


# Step 3: Apply Strategy

# Step 4: Position Account

In [27]:
account = None

In [28]:
tws_conn.register(position_account, message.position)

True

In [29]:
tws_conn.reqPositions()

In [30]:
print(account)

<position account=DU1440139, contract=<ib.ext.Contract.Contract object at 0x7f136554a358>, pos=3500, avgCost=37.59385715>


# Step 5: Placing Orders

In [ ]:
order_id = None

In [ ]:
tws_conn.register(next_id, message.nextValidId)
tws_conn.reqIds(0)

In [26]:
order = create_order(order_type = 'MKT',
                     quantity = 10, 
                     action = 'BUY')

In [27]:
print ('Creating Order: ')
print (' Ticker: ',contract.m_symbol)
print (' SecType: ',contract.m_secType)
print (' Exchange: ',contract.m_exchange)
print (' primExchange: ',contract.m_primaryExch)
print (' Order Quantity: ',order.m_totalQuantity,'\n','Order Type: ', order.m_orderType,'\n','Order Action: ',order.m_action)

Creating Order: 
 Ticker:  AAPL
 SecType:  STK
 Exchange:  SMART
 primExchange:  NASDAQ
 Order Quantity:  10 
 Order Type:  MKT 
 Order Action:  BUY


In [15]:
tws_conn.placeOrder(order_id, contract, order)

Server Error: <error id=62, errorCode=399, errorMsg=Order Message:
BUY 10 AAPL NASDAQ.NMS
Server Error: <error id=62, errorCode=202, errorMsg=Order Canceled - reason:>


# Disconnect

In [58]:
tws_conn.disconnect()

False